# Retrieving Data with ProPublica API

In [14]:
import numpy as np
import pandas as pd
import pickle

import requests
import json
import configparser

from bs4 import BeautifulSoup
import urllib.request
import re

In [ ]:
# Config API
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('propublica', 'PROPUBLICA_API_KEY')

## Retrieve Senators
Through ProPublica, a list of senators can be retrieved for each congress starting with the 80th Congress.  For this project, we will start from the 101st Congress (1989-1990) and end with the 116th Congress (2019-2021).  The API calls will return a response that can be read as a json file.

In [ ]:
# Set parameters
congresses = range(101, 117) # from the 101st congress to the 116th congress

In [ ]:
# Get all congresses
list_of_congresses = []

for n in congresses:
    r = requests.get(
        f'https://api.propublica.org/congress/v1/{n}/senate/members.json',
        headers={'X-API-Key': api_key}
    )
    
    results = r.json()['results'][0]
    members = results['members']
    list_of_congresses.append(members)

In [ ]:
# Function to grab data for senators
def get_senators(members):
    senators = []
    for member in members:
        senator = {
            'id': member['id'],
            'first_name': member['first_name'],
            'last_name': member['last_name'],
            'party': member['party'],
            'gender': member['gender'],
            'state': member['state'],
        }
        senators.append(senator)
    return senators

In [ ]:
senates = []
for members in list_of_congresses:
    senators = get_senators(members)
    senates.append(senators)

In [ ]:
# Create list of all unique senators across 101st to 116th congress
senator_ids = []
all_senators = []
for senate in senates:
    for senator in senate:
        if senator['id'] in senator_ids:
            continue
        else:
            senator_ids.append(senator['id'])
            all_senators.append(senator)

In [ ]:
with open('temp_senators.p', 'rb') as f:
    dict_senators = pickle.load(f)

## Web Scrape Total Roll Call Votes
The API only returns the 20 most recent votes by a senator so searching votes by senator is not possible.  Instead, votes will be retrieve through votes by roll call.  Each senate session (2 sessions per congress) has a varying number of bills voted on so the total number of roll call votes needs to be web scraped per session.  This is done on the [US Senate](https://www.senate.gov/) website.

In [ ]:
# Create BeautifulSoup object to find number of roll call votes for each session
html = urllib.request.urlopen('https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm')
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# Element holding last roll call vote in session
last = soup.find('td').find('a').text
last

In [ ]:
# Last roll call vote in session
re.search(r'^...(?=\\)*', last).group(0)

In [ ]:
# Script to pull total roll call votes per session
roll_calls = {}
for congress in congresses:
    # First session
    html = urllib.request.urlopen(
        f'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{congress}_1.htm'
    )
    soup = BeautifulSoup(html, 'lxml')
    last = soup.find('td').find('a').text
    last_roll_call = re.search(r'^...(?=\\)*', last).group(0)
    roll_calls[f'{congress}_1'] = int(last_roll_call)
    
    # Second session
    html = urllib.request.urlopen(
        f'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_{congress}_2.htm'
    )
    soup = BeautifulSoup(html, 'lxml')
    last = soup.find('td').find('a').text
    last_roll_call = re.search(r'^...(?=\\)*', last).group(0)
    roll_calls[f'{congress}_2'] = int(last_roll_call)

In [ ]:
with open('roll_calls.p', 'rb') as f:
    roll_calls = pickle.load(f)

## Retrieve Votes by Roll Calls
There is a limit of 5000 API calls per day for ProPublica.  So we will make API calls by chunks: 101st-105th Congress (3443 roll calls), 106th-110th Congress (3282), and 11th-116th Congress (3509).  The API call response can be read as a json file.

In [ ]:
# Set range of congresses to make API calls
rc_1 = list(range(101, 106))
rc_2 = list(range(106, 111))
rc_3 = list(range(111, 117))

In [ ]:
# Make API calls for each roll call vote, for each session, for each congress
new_set = []
for key, value in roll_calls.items():
    c = int(key[:3])
    s = int(key[-1])
    if c in rc_3: # Set the range here
        for n in range(1, value+1):
            html = f'https://api.propublica.org/congress/v1/{c}/senate/sessions/{s}/votes/{n}.json'
            r = requests.get(
                html,
                headers={'X-API-Key': api_key}
            )
            new_set.append(r)
    else:
        continue

In [23]:
# Function to extract relevent information from json file
def clean_votes(votes):
    all_votes = []
    for i, v in enumerate(votes):
        try:
            results = v.json()['results']['votes']['vote']
            vote = {
                'congress': results['congress'],
                'session': results['session'],
                'roll_call': results['roll_call'], 
                'bill_id': results['bill']['bill_id'],
                'date': results['date'],
                'positions': results['positions']
            }
        except:
            continue
        all_votes.append(vote)
    return all_votes

In [19]:
with open('c101_105.p', 'rb') as f:
    dict_votes1 = pickle.load(f)
    
with open('c106_110.p', 'rb') as f:
    dict_votes2 = pickle.load(f)
    
with open('c111_116.p', 'rb') as f:
    dict_votes3 = pickle.load(f)

In [29]:
# Merge all roll call votes
all_votes = dict_votes1 + dict_votes2 + dict_votes3